## Import libraries
Import libraries to perform one hot coding and rescaling of features

In [4]:
## Import librariries
import sys, os
import numpy as np
import pandas as pd
from sklearn.preprocessing import OneHotEncoder, StandardScaler, MinMaxScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from joblib import dump, load
import pickle
from typing import List, Tuple
from datetime import datetime
from dateutil import parser

## Import raw features
Import the raw feature file and inspect it.

In [5]:
# Read the raw feature CSV file into a pandas DataFrame
raw_feature_df = pd.read_csv("/Users/adityaponnada/Downloads/time_study_data/raw_features_v1.csv")
print(f"Raw feature DataFrame shape: {raw_feature_df.shape}")
raw_feature_df.head()

Raw feature DataFrame shape: (108075, 20)


/var/folders/h6/3rl340nn7cg92xvwk801_3b80000gn/T/ipykernel_1816/4070788336.py:2: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  raw_feature_df = pd.read_csv("/Users/adityaponnada/Downloads/time_study_data/raw_features_v1.csv")


,participant_id,prompt_time_converted,outcome,is_weekend,time_of_day,in_battery_saver_mode,charging_status,location_category,screen_on,dist_from_home,is_phone_locked,last_phone_usage,wake_day_part,closeness_to_sleep_time,closeness_to_wake_time,mims_5min,days_in_study,completion_24h,completion_since_wake,completion_since_start
0,certifiedembargobartender@timestudy_com,2021-10-23 07:46:02,1,1,Early Morning,0.0,0.0,Transit,1,936.101448,0.0,0.0,0.0,838.966667,1.033333,8.138573,176,0.0,0.0,0.0
1,certifiedembargobartender@timestudy_com,2021-10-23 07:54:01,1,1,Early Morning,NaN,NaN,Transit,1,936.101448,0.0,0.0,0.0,830.983333,9.016667,0.280337,176,1.0,1.0,1.0
2,certifiedembargobartender@timestudy_com,2021-10-23 08:24:00,0,1,Morning,0.0,1.0,Other,0,NaN,1.0,18.1,0.0,801.000000,39.000000,0.000000,176,1.0,1.0,1.0
3,certifiedembargobartender@timestudy_com,2021-10-23 08:32:00,1,1,Morning,0.0,1.0,Transit,1,936.101448,0.0,0.0,0.0,793.000000,47.000000,0.000000,176,1.0,1.0,1.0
4,certifiedembargobartender@timestudy_com,2021-10-23 08:44:04,1,1,Morning,NaN,NaN,Transit,0,936.231251,1.0,9.0,0.0,780.933333,59.066667,160.934583,176,1.0,1.0,1.0


## Explore missing data
For all the features, we will get the % of missing data in them.

In [6]:
def missing_value_table(df):
    skip_cols = ['participant_id', 'prompt_time_converted', 'outcome']
    cols = [col for col in df.columns if col.lower() not in skip_cols]
    missing_percent = df[cols].isnull().mean() * 100
    # Also count empty strings as missing
    empty_percent = (df[cols] == '').mean() * 100
    total_missing_percent = missing_percent + empty_percent
    result = pd.DataFrame({
        'missing_%': total_missing_percent.round(2)
    }).sort_values('missing_%', ascending=False)
    return result
    
# Display missing value table
missing_value_table(raw_feature_df)

,missing_%
in_battery_saver_mode,52.19
charging_status,52.19
dist_from_home,13.88
last_phone_usage,13.80
is_phone_locked,13.80
mims_5min,4.02
wake_day_part,0.04
closeness_to_sleep_time,0.04
closeness_to_wake_time,0.04
completion_since_wake,0.00


In [ ]:
def missing_data_by_participant(df):
    skip_cols = ['prompt_time_converted', 'outcome']
    cols = [col for col in df.columns if col.lower() not in skip_cols and col.lower() != 'participant_id']
    # Create a boolean DataFrame for missing values (NaN or empty string)
    missing_bool = df[cols].isnull() | (df[cols] == '')
    # Group by participant_id and calculate % missing for each column
    missing_percent = missing_bool.groupby(df['participant_id']).mean() * 100
    missing_percent = missing_percent.round(2)
    # Reset index to have participant_id as a column
    missing_percent = missing_percent.reset_index()
    print(missing_percent)
    return missing_percent

# Print % missing data for each participant and variable
missing_data_by_participant(raw_feature_df)

                            participant_id  missing_%  time_of_day  \
0  certifiedembargobartender@timestudy_com        0.0          0.0   
1    chewingslouchingfailing@timestudy_com        0.0          0.0   
2     enjoyergoofinessgrudge@timestudy_com        0.0          0.0   
3  erasuresafeguardravishing@timestudy_com        0.0          0.0   
4        neutergoldfishsworn@timestudy_com        0.0          0.0   
5         persevereriseswoop@timestudy_com        0.0          0.0   
6  pretendedconstrainfraying@timestudy_com        0.0          0.0   
7      remoldexcludingaffair@timestudy_com        0.0          0.0   
8   slapstickporcupineslacks@timestudy_com        0.0          0.0   
9    subtitlegrievousbazooka@timestudy_com        0.0          0.0   

   in_battery_saver_mode  charging_status  location_category  screen_on  \
0           36576.470588     36576.470588                0.0        0.0   
1           41111.764706     41111.764706                0.0        0.0   
2   

/var/folders/h6/3rl340nn7cg92xvwk801_3b80000gn/T/ipykernel_1816/10209776.py:8: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  result = df.groupby('participant_id').apply(percent_missing)


,participant_id,missing_%,time_of_day,in_battery_saver_mode,charging_status,location_category,screen_on,dist_from_home,is_phone_locked,last_phone_usage,wake_day_part,closeness_to_sleep_time,closeness_to_wake_time,mims_5min,days_in_study,completion_24h,completion_since_wake,completion_since_start
0,certifiedembargobartender@timestudy_com,0.0,0.0,36576.470588,36576.470588,0.0,0.0,19770.588235,4747.058824,4747.058824,0.000000,0.000000,0.000000,3223.529412,0.0,0.0,0.0,0.0
1,chewingslouchingfailing@timestudy_com,0.0,0.0,41111.764706,41111.764706,0.0,0.0,11282.352941,78135.294118,78135.294118,0.000000,0.000000,0.000000,2335.294118,0.0,0.0,0.0,0.0
2,enjoyergoofinessgrudge@timestudy_com,0.0,0.0,31752.941176,31752.941176,0.0,0.0,8158.823529,547.058824,547.058824,282.352941,282.352941,282.352941,2035.294118,0.0,0.0,0.0,0.0
3,erasuresafeguardravishing@timestudy_com,0.0,0.0,13523.529412,13523.529412,0.0,0.0,4770.588235,100.000000,100.000000,0.000000,0.000000,0.000000,623.529412,0.0,0.0,0.0,0.0
4,neutergoldfishsworn@timestudy_com,0.0,0.0,27000.000000,27000.000000,0.0,0.0,4082.352941,4205.882353,4205.882353,0.000000,0.000000,0.000000,3070.588235,0.0,0.0,0.0,0.0
5,persevereriseswoop@timestudy_com,0.0,0.0,39335.294118,39335.294118,0.0,0.0,1958.823529,0.000000,0.000000,0.000000,0.000000,0.000000,1729.411765,0.0,0.0,0.0,0.0
6,pretendedconstrainfraying@timestudy_com,0.0,0.0,47305.882353,47305.882353,0.0,0.0,9952.941176,0.000000,0.000000,0.000000,0.000000,0.000000,3623.529412,0.0,0.0,0.0,0.0
7,remoldexcludingaffair@timestudy_com,0.0,0.0,19582.352941,19582.352941,0.0,0.0,3070.588235,11.764706,11.764706,0.000000,0.000000,0.000000,3247.058824,0.0,0.0,0.0,0.0
8,slapstickporcupineslacks@timestudy_com,0.0,0.0,35247.058824,35247.058824,0.0,0.0,8323.529412,0.000000,0.000000,0.000000,0.000000,0.000000,1700.000000,0.0,0.0,0.0,0.0
9,subtitlegrievousbazooka@timestudy_com,0.0,0.0,40382.352941,40382.352941,0.0,0.0,16852.941176,0.000000,0.000000,0.000000,0.000000,0.000000,4000.000000,0.0,0.0,0.0,0.0
